### Test inputs

In [124]:
inputs = [
    '[.##.] (3) (1,3) (2) (2,3) (0,2) (0,1) {3,5,4,7}',
    '[...#.] (0,2,3,4) (2,3) (0,4) (0,1,2) (1,2,3,4) {7,5,12,7,2}',
    '[.###.#] (0,1,2,3,4) (0,3,4) (0,1,2,4,5) (1,2) {10,11,11,5,10,5}'
]

## Task 1

### Parse input

In [125]:
map = {'.': 0, '#': 1}

all_diagrams = []
all_buttons = []
all_joltages = []

for row in inputs:
    tokens = row.split(" ")
    diagram_token = None
    button_tokens = []
    joltage_token = None
    for t in tokens:
        if t.startswith('['):
            diagram_token = t
        elif t.startswith('('):
            button_tokens.append(t)
        elif t.startswith('{'):
            joltage_token = t


    diagram_str = diagram_token.strip('[]')
    diagram = [map[ch] for ch in diagram_str]

    temp_buttons = []
    for t in button_tokens:
        inner = t.strip('()')
        nums = [int(x) for x in inner.split(',')]
        temp_buttons.append(nums)


    joltage_str = joltage_token.strip('{}')
    joltage = [int(x) for x in joltage_str.split(',')]
    
    all_diagrams.append(diagram)
    all_buttons.append(temp_buttons)
    all_joltages.append(joltage)

print(all_diagrams)

[[0, 1, 1, 0], [0, 0, 0, 1, 0], [0, 1, 1, 1, 0, 1]]


In [126]:
# Focus on the first example only
from itertools import combinations
import numpy as np

num_cases = len(all_diagrams)

total_presses = 0
for i in range(num_cases):
    print(i)
    
    diagram = np.array(all_diagrams[i])
    buttons = all_buttons[i]
    num_buttons = len(all_buttons[i])

    flag = False
    num_presses = 1000
    for j in range(len(buttons)):
        comb = list(combinations(buttons, j))
        for c in comb:
            test_case = np.zeros_like(diagram)
            for press in c:
                for val in press:
                    test_case[val] = (test_case[val] + 1) % 2
            if np.all(test_case == diagram):
                flag = True
                num_presses = len(c)
                break
        if flag:
            break
    total_presses += num_presses

0
1
2


In [127]:
buttons

[[0, 1, 2, 3, 4], [0, 3, 4], [0, 1, 2, 4, 5], [1, 2]]

## Task 2

In [128]:
import numpy as np

lhs = np.array((
    (1, 1, 1, 0),
    (1, 0, 0, 1),
    (1, 1, 0, 0),
    (0, 0, 1, 1),
))
rhs = np.array((2, 1, 1, 1))

In [ ]:
from scipy.optimize import linprog


num_presses = 0
# Turn buttons into Matrix
for id, buttons in enumerate(all_buttons):
    
    b = np.array(all_joltages[id])
    
    # Num values and variables
    N_val = len(all_joltages[id])
    N_var = len(buttons)
    A = np.zeros((N_val, N_var))
    
    for i, button in enumerate(buttons):
        A[button, i] = 1
    
    c = np.ones(A.shape[1])
    integrality = np.ones(A.shape[1])
    bounds = [(0, int(max(b))) for i in range(A.shape[1])]
    
    result = linprog(c, A_eq=A, b_eq=b, bounds=bounds, integrality=integrality)
    print(result)
    num_presses += round(result.fun)
    # break

num_presses

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 10.0
              x: [ 1.000e+00  2.000e+00  0.000e+00  4.000e+00  0.000e+00
                   3.000e+00]
            nit: -1
          lower:  residual: [ 1.000e+00  2.000e+00  0.000e+00  4.000e+00
                              0.000e+00  3.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          upper:  residual: [ 6.000e+00  5.000e+00  7.000e+00  3.000e+00
                              7.000e+00  4.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00
                              0.000e+00  0.000e+00]
          eqlin:  residual: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00]
        ineqlin:  residual: []
                 marginals: []
 mip_node_count: 0
 mip

33

In [130]:
from scipy.optimize import Bounds, LinearConstraint, milp


# Turn buttons into Matrix
for id, buttons in enumerate(all_buttons):
    
    b = np.array(all_joltages[id])
    print(b)
    
    # Num values and variables
    N_val = len(all_joltages[id])

    N_var = len(buttons)
    A = np.zeros((N_val, N_var))
    
    
    for i, button in enumerate(buttons):
        A[button, i] = 1
    
    print(A)
    bounds = Bounds(b.min(), b.max())
    
    # # print(slice(0, 9, 1))
    # integrality = np.ones_like(b)
    # # print(integrality)
    sol=milp(A, integrality=integrality, bounds=bounds)
    # print(sol, sum(sol.x))
    # break

# print(A)
# print(b)
# solution = np.linalg.solve(A, b)

[3 5 4 7]
[[0. 0. 0. 0. 1. 1.]
 [0. 1. 0. 0. 0. 1.]
 [0. 0. 1. 1. 1. 0.]
 [1. 1. 0. 1. 0. 0.]]


ValueError: `c` must be a one-dimensional array of finite numbers with at least one element.